# Model Server

In [ ]:
#!python -m pip install v3io

In [1]:
# nuclio: ignore
import nuclio

In [2]:
%nuclio config kind="nuclio:serving"
%nuclio env MODEL_CLASS=ClassifierModel

In [3]:
%nuclio config spec.build.baseImage = "mlrun/ml-serving:0.4.7-py37"

%nuclio: setting spec.build.baseImage to 'mlrun/ml-serving:0.4.7-py37'


In [4]:
%%nuclio cmd -c
python -m pip install v3io

In [5]:
import os
from cloudpickle import load
import kfserving
import numpy as np
from typing import List
from datetime import datetime

In [6]:
import json
import socket
import v3io.dataplane

v3io_client = v3io.dataplane.Client()
hostname = None

def split_path(mntpath=''):
    if mntpath[0] == '/':
        mntpath = mntpath[1:]
    paths = mntpath.split('/')
    container = paths[0]
    subpath = ''
    if len(paths) > 1:
        subpath = mntpath[len(container):]
    return container, subpath
 

def init_stream(stream_path, shards=1):
    global hostname
    hostname = socket.gethostname()
    container, stream_path = split_path(stream_path)
    response = v3io_client.create_stream(container=container,
                                         path=stream_path, 
                                         shard_count=shards,
                                         raise_for_status=v3io.dataplane.RaiseForStatus.never)
    response.raise_for_status([409, 204])
    print(response.__dict__)
    
def push_to_stream(stream_path, data):
    records = [{'data': json.dumps(rec)} for rec in data]
    container, stream_path = split_path(stream_path)
    response = v3io_client.put_records(container=container,
                                       path=stream_path, 
                                       records=records)
    
    #for response_record in response.output.records:
    #    print(response_record.__dict__)

In [7]:
class ClassifierModel(kfserving.KFModel):
    def __init__(self, name: str, model_dir: str, classifier = None):
        super().__init__(name)
        self.model_dir = model_dir
        #self.log_stream = os.environ.get('INFERENCE_STREAM', '')
        #if self.log_stream:
        #    init_stream(self.log_stream)
        if classifier:
            self.classifier = classifier
            self.ready = True

    def load(self):
        """Load model from KubeFlow storage.
        """
        if self.model_dir.endswith('.pkl'):
            model_file = self.model_dir
        else:
            for file in os.path.listdir(self.model_dir):
                if file.endswith('.pkl'):
                    model_file = os.pth.join(self.model_dir, file)
                    break
        self.classifier = load(open(model_file, 'rb'))
        self.ready = True

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample.
        
        :param body : A dict of observations, each of which is an 1-dimensional feature vector.
            
        Returns model predictions as a `List`, one for each row in the `body` input `List`.
        """
        start = datetime.now()
        try:
            feats = np.asarray(body['instances'])
            result: np.ndarray = self.classifier.predict(feats)
            resp = result.tolist()
        except Exception as e:
            raise Exception(f"Failed to predict {e}")
        
#         if self.log_stream:
#             data = {'request': body, 'resp': resp, 
#                     'class': 'ClassifierModel', 
#                     'model': self.name,
#                     'host': hostname,
#                     'when': str(start), 
#                     'microsec': (datetime.now()-start).microseconds}
#             push_to_stream(self.log_stream, [data])

        return resp

In [8]:
# nuclio: end-code

# save and deploy

In [9]:
from mlrun import new_model_server
from mlrun import mount_v3io, mlconf
import os

# there will be 3 models from sklearn-pipe ... let's assume that the models 
# are stored at this location and that we want to serve all 3 (for whatever reason)
# ot sure how to manage the new inference stream in this scenario

for model in os.listdir(mlconf.artifact_path + '/sklearn_classifier/models'):
    if model.endswith("pkl"):
        model = model.split('.')[-2]
        fn = new_model_server(f'iris-{model}', model_class='ClassifierModel')
        fn.spec.description = f"generic sklearn {model} model server"
        fn.metadata.categories = ['serving', 'models', f'{model}']
        fn.metadata.labels = {'author': 'yaronh'}
        fn.save()
        #print(fn.to_yaml())
        fn.export(f"{model}/function.yaml")
        fn.apply(mount_v3io())
        fn.set_envs({f"SERVING_MODEL_iris_{model}": mlconf.artifact_path + f"/sklearn_classifier/models/{model}.pkl",
                     'INFERENCE_STREAM': '/User/admin/tststream'})
        fn.deploy(project='sklearn-servers')

[mlrun] 2020-04-29 23:46:28,161 saving function: iris-adaboostclassifier, tag: latest
[mlrun] 2020-04-29 23:46:28,192 function spec saved to path: function.yaml
[mlrun] 2020-04-29 23:46:28,193 deploy started
[nuclio] 2020-04-29 23:46:28,266 (info) Building processor image
[nuclio] 2020-04-29 23:46:31,296 (info) Build complete
[nuclio] 2020-04-29 23:47:33,844 (error) Failed to deploy. Details:
[repeated 6 times] [E 200429 23:47:29 logger:95] Exception raised while running init_context
Exception raised while running init_context [worker_id="0"]
[E 200429 23:47:29 logger:95] Exception raised while running init_context
[repeated 4 times] [E 200429 23:47:29 logger:107] Caught unhandled exception while initializing
Caught unhandled exception while initializing [err="Request failed with status 404: b'{
	"ErrorCode": -16777233,
	"ErrorMessage": "Container not found"
}'" || traceback="Traceback (most recent call last):
  File "/opt/nuclio/_nuclio_wrapper.py", line 283, in run_wrapper
    wrappe

DeployError: cannot deploy 